## 

### Alterações no script tester.py

* Adicionar parênteses no ```print``` do script ```tester.py```
* alterar linha de import de biblioteca ```from sklearn.cross_validation import StratifiedShuffleSplit``` para ```from sklearn.model_selection import StratifiedShuffleSplit```

### 
* Copiar ```feature_format.py``` para pasta local

### Alterações feitas no poi_id.py
* alterar chamada ```with open()``` para usar o segundo para parâmetro como ```rb``` 

### Estudar aplicação de NMF (Non-negative Matrix Factorization) para o projeto

### Ver aplicação de log loss

### Orientações na hora de escolher um modelo

* Começar sempre com um modelo mais simples e explorar os dados e features
* Começar com variáveis númericas
* Usar regressão Logistica multi classe
    * Treinar o classificador em cada label separadamente e ver desempenho na predição
* Como POI são raros, usar StratifiedShuffleSplit
* OneVsRestClassifier trata cada coluna de y de forma independente e determina um classificdor  para cada coluna

### Usando a função de pipeline do sklearn
* Usar ```imputer``` para lidar com dados faltantes



In [24]:
import pandas as pd
import sys
import pickle
from tester import dump_classifier_and_data
from feature_format import featureFormat, targetFeatureSplit

### Para trabalhar com features com informação de texto e numérica no mesmo modelo
* Deve-se dividir o processamento de dados numéricos e texto usando ```FunctionTransformer``` e depois ```FeatureUnion``` em ambos os datasets

In [ ]:
# Import the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Split out only the text data
X_train, X_test, y_train, y_test = train_test_split(sample_df['text'],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=456)

# Instantiate Pipeline object: pl
pl = Pipeline([
        ('vec', CountVectorizer),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - just text data: ", accuracy)

# Import FunctionTransformer
from sklearn.preprocessing import FunctionTransformer

# Obtain the text data: get_text_data
get_text_data = FunctionTransformer(lambda x: x['text'], validate=False)

# Obtain the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[['numeric', 'with_missing']], validate=False)

# Fit and transform the text data: just_text_data
just_text_data = get_text_data.fit_transform(sample_df)

# Fit and transform the numeric data: just_numeric_data
just_numeric_data = get_numeric_data.fit_transform(sample_df)

# Print head to check results
print('Text Data')
print(just_text_data.head())
print('\nNumeric Data')
print(just_numeric_data.head())

In [ ]:
# Import FeatureUnion
from sklearn.pipeline import FeatureUnion

# Split using ALL data in sample_df
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric', 'with_missing', 'text']],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=22)

# Create a FeatureUnion with nested pipeline: process_and_join_features
process_and_join_features = FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )

# Instantiate nested pipeline: pl
pl = Pipeline([
        ('union', process_and_join_features),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])


# Fit pl to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - all data: ", accuracy)

In [ ]:
# Import FunctionTransformer
from sklearn.preprocessing import FunctionTransformer

# Get the dummy encoding of the labels
dummy_labels = pd.get_dummies(df[LABELS])

# Get the columns that are features in the original df
NON_LABELS = [c for c in df.columns if c not in LABELS]

# Split into training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(df[NON_LABELS],
                                                               dummy_labels,
                                                               0.2, 
                                                               seed=123)

# Preprocess the text data: get_text_data
get_text_data = FunctionTransformer(combine_text_columns, validate=False)

# Preprocess the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS], validate=False)


In [ ]:
# Complete the pipeline: pl
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on budget dataset: ", accuracy)

In [ ]:
# Import random forest classifer
from sklearn.ensemble import RandomForestClassifier

# Edit model step in pipeline
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('clf', RandomForestClassifier())
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on budget dataset: ", accuracy)

In [6]:
features_list = ['poi','salary'] # You will need to use more features

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)


In [22]:
# Checando chaves de cada entrada no dicionário
list(data_dict.get('ALLEN PHILLIP K').keys())

['salary',
 'to_messages',
 'deferral_payments',
 'total_payments',
 'loan_advances',
 'bonus',
 'email_address',
 'restricted_stock_deferred',
 'deferred_income',
 'total_stock_value',
 'expenses',
 'from_poi_to_this_person',
 'exercised_stock_options',
 'from_messages',
 'other',
 'from_this_person_to_poi',
 'poi',
 'long_term_incentive',
 'shared_receipt_with_poi',
 'restricted_stock',
 'director_fees']

In [27]:
df_enron = pd.DataFrame.from_dict(data_dict, orient='index')

In [1]:
df_enron[df_enron.poi ==True].shape

NameError: name 'df_enron' is not defined

In [31]:
df_enron.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 21 columns):
salary                       146 non-null object
to_messages                  146 non-null object
deferral_payments            146 non-null object
total_payments               146 non-null object
loan_advances                146 non-null object
bonus                        146 non-null object
email_address                146 non-null object
restricted_stock_deferred    146 non-null object
deferred_income              146 non-null object
total_stock_value            146 non-null object
expenses                     146 non-null object
from_poi_to_this_person      146 non-null object
exercised_stock_options      146 non-null object
from_messages                146 non-null object
other                        146 non-null object
from_this_person_to_poi      146 non-null object
poi                          146 non-null bool
long_term_incentive          146 non-null object


In [33]:
list(df_enron.salary)

[201955,
 'NaN',
 477,
 267102,
 239671,
 80818,
 231330,
 213999,
 'NaN',
 216582,
 187922,
 'NaN',
 213625,
 248546,
 'NaN',
 278601,
 'NaN',
 248017,
 261516,
 330546,
 240189,
 261809,
 415189,
 'NaN',
 'NaN',
 'NaN',
 288542,
 'NaN',
 314288,
 184899,
 206121,
 365163,
 492375,
 210500,
 250100,
 262788,
 221003,
 278601,
 'NaN',
 210692,
 182245,
 170941,
 304588,
 440698,
 199157,
 'NaN',
 'NaN',
 1060932,
 'NaN',
 192008,
 231946,
 'NaN',
 'NaN',
 'NaN',
 274975,
 272880,
 'NaN',
 6615,
 374125,
 243293,
 'NaN',
 'NaN',
 'NaN',
 262663,
 211788,
 'NaN',
 'NaN',
 'NaN',
 130724,
 85274,
 288558,
 'NaN',
 275101,
 404338,
 174246,
 271442,
 309946,
 224305,
 339288,
 1072321,
 273746,
 'NaN',
 'NaN',
 236457,
 'NaN',
 'NaN',
 349487,
 'NaN',
 263413,
 365038,
 'NaN',
 370448,
 'NaN',
 365788,
 'NaN',
 'NaN',
 'NaN',
 267093,
 251654,
 229284,
 'NaN',
 329078,
 94941,
 261879,
 'NaN',
 655037,
 197091,
 'NaN',
 'NaN',
 'NaN',
 96840,
 76399,
 420636,
 249201,
 'NaN',
 'NaN',
 3041